_Load libraries_

In [1]:
import pandas as pd
import numpy as np
import itertools as it

_Load csv files_

In [15]:
loans_lender = pd.read_csv("D:\\Università\\Magistrale\\Primo anno\\Foundation of computer science\\2019-20\\Progetto\\additional-kiva-snapshot\\loans_lenders.csv",
                           engine='python')

### 1. Normalize the loan_lenders table. In the normalized table, each row must have one loan_id and one lender.

In [16]:
loans_lender["lenders"]=loans_lender["lenders"].str.split(',') #divisione lenders
loans_lender_norm=loans_lender.explode('lenders')

,loan_id,lenders
0,483693,muc888
1,483693,sam4326
2,483693,camaran3922
3,483693,lachheb1865
4,483693,rebecca3499
...,...,...
28293926,1206425,trogdorfamily7622
28293927,1206425,danny6470
28293928,1206425,don6118
28293929,1206486,alan5175


In [18]:
loans_lender_norm=loans_lender_norm.reset_index(drop=True)
loans_lender_norm.head()

,loan_id,lenders
0,483693,muc888
1,483693,sam4326
2,483693,camaran3922
3,483693,lachheb1865
4,483693,rebecca3499
